# HMM
    Hidden Markov Model

[](https://www.cs.sjsu.edu/~stamp/RUA/HMM.pdf)

## Notation
    - A: state transition matrix
    - B: observation probability / emission matrix
    - O: observation sequence
    - PI: initial state

In [53]:
import numpy as np

A = np.array([
    [0.7, 0.3], 
    [0.4, 0.6]])
B = np.array([
    [0.1, 0.4, 0.5], 
    [0.7, 0.2, 0.1]])
O = np.array([
    0,
    1,
    0,
    2])
PI = np.array([[0.6, 0.4]])

## Problem 1
    An Evaluation Problem

### Description
    given A,B,PI and O, compute p
    
### Solution
    forward algorithm

In [54]:
# 计算前向变量矩阵 alpha
def forward(a,b,pi,o):
    T = len(o)
    N = a.shape[0]
    alpha = np.zeros((T,N))
    alpha[0] = pi * b[:, o[0]]
#     print('initial alpha \n', alpha, '\n')
    
    for t in range(1,T):
        alpha[t] = alpha[t-1].dot(a) * b[:, o[t]]
#         print('alpha at step '+str(t) + '\n', alpha, '\n')
    return alpha

# marginalized likelihood
def forward_probability(a,b,pi,o):
    return forward(a,b,pi,o)[-1].sum()

print('alpha \n', forward(A,B,PI,O), '\n')
print('forward probability', forward_probability(A,B,PI,O))

alpha 
 [[0.06      0.28     ]
 [0.0616    0.0372   ]
 [0.0058    0.02856  ]
 [0.007742  0.0018876]] 

forward probability 0.009629599999999997


## Problem 2
    Decoding / Prediction Problem

### Description
    given A,B,PI,O, find optimal state sequence
    
### Solution
    viterbi algorithm,  a DP algorithm
    should yield same result as that from brute force by compute p of all possible path with forward algorithm

In [56]:
def viterbi(a,b,pi,o):
    T = len(o)
    N = a.shape[0]
    delta = np.zeros((T,N))
    psi = np.zeros((T,N))
    delta[0] = pi * b[:, o[0]]
#     print('init delta\n', delta, '\n' )
#     print('init psi\n', psi, '\n' )    
    
    for t in range(1,T):
        for i in range(N):
            delta[t,i] = np.max(delta[t-1]* a[:,i]) * b[i, o[t]] 
            psi[t,i] = np.argmax(delta[t-1]* a[:,i])
#         print('step ' + str(t) + ' delta\n', delta, '\n' )
#         print('step ' + str(t) + ' psi\n', psi, '\n' )
    print('delta\n', delta, '\n' )
    print('psi\n', psi, '\n' )    
    
    # backtrack
    i_star = np.zeros(T,dtype=np.int32)
    i_star[T-1] = np.argmax(delta[T-1])
    for t in range(T-2,-1,-1): 
        i_star[t] = psi[t+1, i_star[t+1]]
    
    return i_star

print('viterbi\n', viterbi(A,B,PI,O))


delta
 [[0.06       0.28      ]
 [0.0448     0.0336    ]
 [0.003136   0.014112  ]
 [0.0028224  0.00084672]] 

psi
 [[0. 0.]
 [1. 1.]
 [0. 1.]
 [1. 1.]] 

viterbi
 [1 1 1 0]


The second row of psi should be [1,1], which means that 
1. the best path of length two ending with 0 is 1->0 with p=0.0448
2. the best path of length two ending with 1 is 1->1 with p=0.0336

## Problem 3
    Learning Problem

### Description
    given O, total number of distinct states(A.shape), total number of distinct observations(B.shape), find optimal state sequence
    
### Solution
    forward-backward algorithm, aka Baum-Welch algorithm
    it is a particular case of EM(Expectation-Maximization) algorithm

```
Unsupervised training of a HMM involves running forward and backward to estimate the *expected* probability of taking various state sequences, then updates the model to match these *expectations*. This repeats many times until things stabilise (covergence). Note that it's non-convex, so the starting point often affects the converged solution. --from https://people.eng.unimelb.edu.au/tcohn/comp90042/HMM.py
```

In [57]:
# reuse forward function

# 计算后向变量矩阵 beta
def backward(a,b,pi,o):
    T = len(o)
    N = a.shape[0]
    beta = np.zeros((T,N))
    beta[-1:,:] = 1
#     print('initial beta \n', beta, '\n') 
    
    for t in range(T-2,-1,-1):
        for i in range(N):
            beta[t,i] = np.sum( a[i, :] * b[:, o[t+1]] * beta[t+1,:])
#         print('beta at step '+str(T-t-1) + '\n', beta, '\n')

    return beta

# marginalized likelihood
def backward_probability(a,b,pi,o):
    beta = backward(a,b,pi,o)
    return np.sum(pi * b[:,o[0]] * beta[0,:])

# 计算 posterior probability
# 每行 sum = 1
def gamma(a,b,pi,o):
    alpha = forward(a,b,pi,o)
    beta = backward(a,b,pi,o)
    p = alpha[-1].sum()
    return np.multiply(alpha, beta) / p

print('beta \n', backward(A,B,PI,O), '\n')
print('backward probability \n', backward_probability(A,B,PI,O), '\n')
print('should be equal to forward probability \n', forward_probability(A,B,PI,O), '\n')

print('gamma \n', gamma(A,B,PI,O), '\n')

beta 
 [[0.0302  0.02792]
 [0.0812  0.1244 ]
 [0.38    0.26   ]
 [1.      1.     ]] 

backward probability 
 0.009629599999999999 

should be equal to forward probability 
 0.009629599999999997 

gamma 
 [[0.18816981 0.81183019]
 [0.51943175 0.48056825]
 [0.22887763 0.77112237]
 [0.8039794  0.1960206 ]] 

